In [1]:
if 'google.colab' in str(get_ipython()):
    print("Running on Colab")
    from google.colab import drive, output
    drive.mount('/content/drive')
    import sys
    !pip install torchaudio
    !pip install pythae
    output.clear()
    #%pdb on

    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
else:
    print("Running local")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform
import pythae
from Modules.Utils import compute_size
from pythae.pipelines import TrainingPipeline
from pythae.models import VQVAE, VQVAEConfig
from pythae.models import VAE, VAEConfig
from pythae.models import VAE_LinNF, VAE_LinNF_Config
from pythae.models import AE, AEConfig
from pythae.models import MIWAE, MIWAEConfig
from pythae.trainers import BaseTrainerConfig
from pythae.models.nn import BaseDecoder
from pythae.models.base.base_utils import ModelOutput
from pythae.models.nn import BaseEncoder
from AE_Clustering import AE_Clustering 
device = torch.device("cuda")

from Pythae_DataLoader import SoundscapeData
# from Pythae_Models import My_encoder
# from Pythae_Models import My_decoder


Running local


In [2]:
compute_size((515,2574), [(3,3),(3,3),(3,3),(2,2)], [(8,8),(8,8),(4,4),(2,2)], [(0,0),(0,0),(0,2),(0,1)],1)

([(515, 2574), (170, 856), (55, 283), (18, 94), (9, 47)],
 [(9, 47), (18, 94), (55, 283), (170, 856), (515, 2574)])

In [3]:
class My_encoder(BaseEncoder):

    def __init__(self,args):
        BaseEncoder.__init__(self)
        self.encoder = nn.Sequential(
            nn.Conv2d(1, args.latent_dim // 8, kernel_size=8, stride=3, padding=0),  # N, 256, 127, 8004
            nn.ReLU(),
            nn.Conv2d(args.latent_dim // 8, args.latent_dim // 4, kernel_size=8, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(args.latent_dim // 4, args.latent_dim // 2, kernel_size=4, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(args.latent_dim // 2, args.latent_dim, kernel_size=2, stride=2, padding=0),  # N, 512, 125,969
            nn.ReLU()
        ) 

    def forward(self, x: torch.Tensor):
        embedding = self.encoder(x)
        output = ModelOutput(
            embedding=embedding,
            # log_covariance=log_var # for VAE based models
        )
        return output


class My_decoder(BaseDecoder):

    def __init__(self, args):
        BaseDecoder.__init__(self)
        self.decoder = nn.Sequential(  # This is like go in opposite direction respect the encoder
            nn.ConvTranspose2d(args.latent_dim, args.latent_dim // 2, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(args.latent_dim // 2, args.latent_dim // 4, kernel_size=4, stride=3, padding=0,
                               output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(args.latent_dim // 4, args.latent_dim // 8, kernel_size=8, stride=3, padding=0,
                               output_padding=(0,2)),
            nn.ReLU(),
            nn.ConvTranspose2d(args.latent_dim // 8, 1, kernel_size=8, stride=3, padding=0, output_padding=(0,1)),
            nn.Sigmoid()
        )

    def forward(self, z: torch.Tensor):
           reconstruction = self.decoder(z)
           output = ModelOutput(
                reconstruction=reconstruction
            )
           return output


In [4]:
my_training_config = BaseTrainerConfig(
    output_dir='my_model_AE',
    num_epochs=3,
    learning_rate=1e-3,
    batch_size=14,
    steps_saving=1
)

In [5]:
my_ae_config = model_config = AEConfig(
    input_dim=(1, 515, 2574),
    latent_dim=64,
    uses_default_decoder= False,
    uses_default_encoder= False

)



In [6]:
encoder = My_encoder(model_config)
decoder= My_decoder(model_config)

In [7]:
my_ae_model = AE(
    model_config=my_ae_config,
    encoder = encoder,
    decoder = decoder
).to(device)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [8]:
pipeline = TrainingPipeline(
    training_config=my_training_config,
    model=my_ae_model
)

dataset_test = torch.load(f'temporal/pythae_dataset_test_ae_jaguas')
dataset_train = torch.load(f'temporal/pythae_dataset_train_ae_jaguas')

In [9]:
root_path = '/content/drive/Shareddrives/ConservacionBiologicaIA/Datos/Jaguas_2018'
root_path = '/media/mirp_ai/DATA1/Jaguas_2018'

dataset = SoundscapeData(root_path, audio_length=60, ext="wav", win_length=1028)

dataset_train, dataset_test = random_split(dataset,
                                           [round(len(dataset)*0.2), len(dataset) - round(len(dataset)*0.2)], 
                                           generator=torch.Generator().manual_seed(1024))

# dataset_small_test, dataset_big_test = random_split(dataset_test,
#                                            [round(len(dataset_test)*0.1), len(dataset_test) - round(len(dataset_test)*0.1)], 
#                                            generator=torch.Generator().manual_seed(1024))

# torch.save(dataset_test, "temporal/pythae_dataset_test_ae_jaguas")
# torch.save(dataset_train, "temporal/pythae_dataset_train_ae_jaguas") 

In [ ]:
pipeline(
    train_data=dataset_train, # must be torch.Tensor or np.array
    eval_data=dataset_test# must be torch.Tensor or np.array
)


Checking train dataset...
Checking eval dataset...
Using Base Trainer

Model passed sanity check !
Ready for training.

Created my_model_AE/AE_training_2023-03-22_12-36-10. 
Training config, checkpoints and final model will be saved here.

Training params:
 - max_epochs: 3
 - per_device_train_batch_size: 64
 - per_device_eval_batch_size: 64
 - checkpoint saving every: 1
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)
Scheduler: None

Successfully launched training !



Training of epoch 1/3:   0%|          | 0/220 [00:00<?, ?batch/s]

## Using the trained model and samplers

In [ ]:
import os
from pythae.models import AutoModel

In [ ]:
last_training = sorted(os.listdir("my_model_AE"))[-1]
print(last_training)
trained_model = AutoModel.load_from_folder(os.path.join("my_model_AE", last_training, "final_model"))

In [ ]:
trained_model = trained_model.to("cpu")

In [ ]:
training_loader = DataLoader(dataset_train, batch_size=100)
test_loader = DataLoader(dataset_test, batch_size=100)
iterator = iter(test_loader)

In [ ]:
dictado = next(iterator)
type(dictado.dat a)
dictado.data.shape

In [ ]:
a = trained_model.encoder(dictado.data)
a[0][0]
print((a[0].shape))
b = trained_model.decoder(a[0])
b[0].shape

In [ ]:
import matplotlib.pyplot as plt
num=2
im=41
plt.figure(figsize=(3,3))
plt.imshow(dictado.data[im,0,:],vmin=0, vmax=0.2)
plt.savefig(f"pythae_ori_{num}")
c = b[0].detach().numpy()
plt.figure(figsize=(3,3))
plt.imshow(c[im,0,:],vmin=0, vmax=0.2)
plt.savefig(f"pythae_rec_{num}")


# Pythae Clustering

In [ ]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=self._n_clusters, random_state=0)
for id, item in enumerate(self._dataset):
    if id+1 == 3:
        break
    else:
        pass
    print(f"id: {id+1} of {len(self._dataset)}")
    trained_model = trained_model.to("cuda")
    originals = next(iterator)
    encodings = trained_model.encoder(originals.data)
    labels = originals.labels

    encodings_size = encodings[0].shape
    labels = labeling(label, repetitions=4, axes=0)

    encodings = encodings.to("cpu").detach()
    encodings = encodings.reshape(encodings.shape[0],
                                encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
    kmeans = kmeans.partial_fit(encodings)
    # embedding = self.kmeans.transform(encodings)
    # mbk_means_cluster_centers = self.kmeans.cluster_centers_
    # mbk_means_labels = pairwise_distances_argmin(encodings, mbk_means_cluster_centers)
    mbk_means_labels = self.kmeans.predict(encodings)
    X_embedded_TSNE = TSNE(n_components=2, learning_rate='auto',
                            init='random', random_state=0).fit_transform(encodings)
    # reducer = umap.UMAP()
    # X_embedded_UMAP = reducer.fit_transform(encodings)

    silhouette_score_TSNE.append(metrics.silhouette_score(X_embedded_TSNE, mbk_means_labels))
    print(silhouette_score_TSNE[id])
    if id+1 % plot_clusters_period == 0:
        self.plot_clusters(X_embedded_TSNE, mbk_means_labels, labels)
        # self.plot_clusters(X_embedded_UMAP, mbk_means_labels, labels)
    if id+1 % 2 == 0:
        self.plot_silhouette(encodings, mbk_means_labels, self._n_clusters, silhouette_score_TSNE[id])

## Using wandb logging

In [ ]:
from pythae.trainers.training_callbacks import WandbCallback

callbacks = [] # the TrainingPipeline expects a list of callbacks

wandb_cb = WandbCallback() # Build the callback 

# SetUp the callback 
wandb_cb.setup(
    training_config=training_config, # training config
    model_config=model_config, # model config
    project_name="your_wandb_project", # specify your wandb project
    entity_name="your_wandb_entity", # specify your wandb entity
)

callbacks.append(wandb_cb) # Add it to the callbacks list